# Tools

This notebook contains all the tools that will be used by the agent.

In [1]:
from dotenv import load_dotenv
import os
import requests
import sqlite3
import pandas as pd

# For AI Agent
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
import re
import requests
from smolagents import CodeAgent, OpenAIServerModel, tool, Tool
from markdownify import markdownify
from requests.exceptions import RequestException

# For internet searches
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

# For LLMs
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig

# Get top headlines

In [2]:
# Get a visit webpage tool
@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [3]:
# Test the visit_webpage tool
print(visit_webpage("https://warhammer40k.fandom.com/wiki/Imperium_of_Man"))

Imperium of Man | Warhammer 40k Wiki | Fandom

[Sign In](https://auth.fandom.com/signin?source=mw&redirect=https%3A%2F%2Fwarhammer40k.fandom.com%2Fwiki%2FImperium_of_Man)

[Register](https://auth.fandom.com/register?source=mw&redirect=https%3A%2F%2Fwarhammer40k.fandom.com%2Fwiki%2FImperium_of_Man)

[![Warhammer 40k Wiki](https://static.wikia.nocookie.net/warhammer40k/images/e/e6/Site-logo.png/revision/latest?cb=20210625083502)](https://warhammer40k.fandom.com)
[Warhammer 40k Wiki](https://warhammer40k.fandom.com)

* [Explore](#)

  + [Main Page](https://warhammer40k.fandom.com/wiki/Warhammer_40k_Wiki)
  + [Discuss](/f)
  + [All Pages](https://warhammer40k.fandom.com/wiki/Special:AllPages)
  + [Community](https://warhammer40k.fandom.com/wiki/Special:Community)
  + [Interactive Maps](https://warhammer40k.fandom.com/wiki/Special:AllMaps)
  + [Recent Blog Posts](/Blog:Recent_posts)
* [Imperium of Man](https://warhammer40k.fandom.com/wiki/Category:Imperium)

  + [Imperial History](https://w

In [4]:
# Create the agent to use the tool
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    InferenceClientModel,
    WebSearchTool,
    LiteLLMModel,
    OpenAIServerModel
)

model_id = "gpt-4.1-mini"

model = OpenAIServerModel(model_id=model_id, api_key=OPENAI_API_KEY)

web_agent = ToolCallingAgent(
    tools=[WebSearchTool(), visit_webpage],
    model=model,
    max_steps=10,
    name="web_search_agent",
    description="Runs web searches for you.",
)

In [5]:
# Create the manager agent to manage the web agent
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent],
    additional_authorized_imports=["time", "numpy", "pandas"],
)

In [6]:
# Test the functionality of the manager agent
answer = manager_agent.run("Get the most viewed Hearthstone Battlegrounds news today, and format it into an array", max_steps=3)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Get the most viewed Hearthstone Battlegrounds news today, and format it into an array                           │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4.1-mini ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task_description = (                                                                                             
      "Find the most viewed Hearthstone Battlegrounds news today. "                                                
      "Extract the title and a brief summary or first sentence for each news article. "                            
      "Format the results into an array in the format: "                                                           
      "[{'title': title1, 'summary': summary1}, {'title': title2, 'summary': summary2}, ...]. "                    
      "Return only the array without extra text."                                                                  
  )                                                                                                                
                                                                                                                   
  search_results = web_search_agent(task=task_description, additional_args={})                                     
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────────── New run - web_search_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'web_search_agent'.                                                                │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Find the most viewed Hearthstone Battlegrounds news today. Extract the title and a brief summary or first       │
│ sentence for each news article. Format the results into an array in the format: [{'title': title1, 'summary':   │
│ summary1}, {'title': title2, 'summary': summary2}, ...\]. Return only the array without extra text.             │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4.1-mini ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 22.02 seconds| Input tokens: 2,069 | Output tokens: 172]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  news_array = [                                                                                                   
    {                                                                                                              
      'title': 'Hearthstone Battlegrounds Has Launched Season 11',                                                 
      'summary': 'Hearthstone Battlegrounds Season 11: Echoes of Un\'Goro launches with new minions, spells, and   
  features. Quests return with 48 total options, bringing 13 new and 35 classic quests for fresh strategies.       
  Introducing the Rally keyword, activating unique effects when Rally minions attack in battle. New Guide feature  
  and Loh, the Living Legend hero, join refreshed rewards and Season Pass cosmetics.'                              
    },                                                                                                             
    {                                                                                                              
      'title': '33.2 Patch Notes: New Battleground Season & Constructed Changes',                                  
      'summary': 'Explore the Lost City of Un\'Goro with the new Rally keyword, the return of Quests, a major      
  minion refresh, a new Guide feature, Dual-Class and Heroic Tavern Brawls, a Hearthstone balance update, and      
  more! All in Battlegrounds Season 11: Echoes of Un\'Goro, going live August 5 at 10:00 a.m. PT.'                 
    },                                                                                                             
    {                                                                                                              
      'title': 'Hearthstone Update 32.2.2 Brings Mini-Set Embers of the World Tree and Major Battlegrounds         
  Changes',                                                                                                        
      'summary': 'On May 13, 2025, Blizzard released Hearthstone update 32.2.2, featuring the new mini-set         
  "Embers of the World Tree" and extensive balance changes to the Battlegrounds mode.'                             
    },                                                                                                             
    {                                                                                                              
      'title': 'Hearthstone: Battlegrounds Season 10 "Second Nature" Launches with Major Updates',                 
      'summary': 'On April 29, 2025, Blizzard released the long-awaited 10th season of Hearthstone\'s              
  Battlegrounds mode, titled Second Nature.'                                                                       
    },                                                                                                             
    {                                                                                                              
      'title': 'Hearthstone Unveils Embers of the World Tree Mini-Set',                                            
      'summary': 'Blizzard unveiled the "Embers of the World Tree" mini-set for Hearthstone, adding new cards and  
  gameplay elements to the game.'                                                                                  
    },                                                                                                             
    {                                                                                                              
      'title': 'Hearthstone Announces New Expansion: The Lost City of Un\'Goro',                                   
      'summary': 'Blizzard announced a new Hearthstone expansion, The Lost City of Un\'Goro, featuring new         
  mechanics and content.'                                

Final answer: [{'title': 'Hearthstone Battlegrounds Has Launched Season 11', 'summary': "Hearthstone Battlegrounds 
Season 11: Echoes of Un'Goro launches with new minions, spells, and features. Quests return with 48 total options, 
bringing 13 new and 35 classic quests for fresh strategies. Introducing the Rally keyword, activating unique 
effects when Rally minions attack in battle. New Guide feature and Loh, the Living Legend hero, join refreshed 
rewards and Season Pass cosmetics."}, {'title': '33.2 Patch Notes: New Battleground Season & Constructed Changes', 
'summary': "Explore the Lost City of Un'Goro with the new Rally keyword, the return of Quests, a major minion 
refresh, a new Guide feature, Dual-Class and Heroic Tavern Brawls, a Hearthstone balance update, and more! All in 
Battlegrounds Season 11: Echoes of Un'Goro, going live August 5 at 10:00 a.m. PT."}, {'title': 'Hearthstone Update 
32.2.2 Brings Mini-Set Embers of the World Tree and Major Battlegrounds Changes', 'summary': 'On May 13, 2025, 
Blizzard released Hearthstone update 32.2.2, featuring the new mini-set "Embers of the World Tree" and extensive 
balance changes to the Battlegrounds mode.'}, {'title': 'Hearthstone: Battlegrounds Season 10 "Second Nature" 
Launches with Major Updates', 'summary': "On April 29, 2025, Blizzard released the long-awaited 10th season of 
Hearthstone's Battlegrounds mode, titled Second Nature."}, {'title': 'Hearthstone Unveils Embers of the World Tree 
Mini-Set', 'summary': 'Blizzard unveiled the "Embers of the World Tree" mini-set for Hearthstone, adding new cards 
and gameplay elements to the game.'}, {'title': "Hearthstone Announces New Expansion: The Lost City of Un'Goro", 
'summary': "Blizzard announced a new Hearthstone expansion, The Lost City of Un'Goro, featuring new mechanics and 
content."}]

[Step 2: Duration 7.20 seconds| Input tokens: 5,165 | Output tokens: 710]

In [7]:
# Print the answer
print(answer)

[{'title': 'Hearthstone Battlegrounds Has Launched Season 11', 'summary': "Hearthstone Battlegrounds Season 11: Echoes of Un'Goro launches with new minions, spells, and features. Quests return with 48 total options, bringing 13 new and 35 classic quests for fresh strategies. Introducing the Rally keyword, activating unique effects when Rally minions attack in battle. New Guide feature and Loh, the Living Legend hero, join refreshed rewards and Season Pass cosmetics."}, {'title': '33.2 Patch Notes: New Battleground Season & Constructed Changes', 'summary': "Explore the Lost City of Un'Goro with the new Rally keyword, the return of Quests, a major minion refresh, a new Guide feature, Dual-Class and Heroic Tavern Brawls, a Hearthstone balance update, and more! All in Battlegrounds Season 11: Echoes of Un'Goro, going live August 5 at 10:00 a.m. PT."}, {'title': 'Hearthstone Update 32.2.2 Brings Mini-Set Embers of the World Tree and Major Battlegrounds Changes', 'summary': 'On May 13, 2025,

# Determine Priority

In [8]:
# TODO Determine Priority


# Summarises the headline

In [9]:
# Define Model
summariser_model_name = 'google/flan-t5-base'


In [10]:
# Get text to summarise (Accepts (human-readable) string as input)
text_to_summarise = "The Imperium of Man, also called simply the Imperium, is a galaxy-spanning, interstellar Human empire, the ultimate authority for the vast majority of the Human species in the Milky Way galaxy in the 41st Millennium A.D. It is ruled by the living god who is known as the Emperor of Mankind. However, there are other humanoid species classified as Imperial citizens, mainly mutant offshoots of genetic base-line Humans who are known as Abhumans and include such Human sub-species as the Ogryns, Ratlings and Squats. The founder and ruler of the Imperium is the god-like Emperor of Mankind, the most powerful Human psyker ever born. The Emperor founded the Imperium over 10,000 Terran years ago in the late 30th Millennium during the Unification Wars on Old Earth following the apocalyptic period in Human history known as the Age of Strife."


In [11]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(summariser_model_name)
summariser_model = AutoModelForSeq2SeqLM.from_pretrained(summariser_model_name)


In [12]:
# Create the prompt
summariser_prompt = f'Write a short summary for this text: {text_to_summarise}'
print(summariser_prompt)


Write a short summary for this text: The Imperium of Man, also called simply the Imperium, is a galaxy-spanning, interstellar Human empire, the ultimate authority for the vast majority of the Human species in the Milky Way galaxy in the 41st Millennium A.D. It is ruled by the living god who is known as the Emperor of Mankind. However, there are other humanoid species classified as Imperial citizens, mainly mutant offshoots of genetic base-line Humans who are known as Abhumans and include such Human sub-species as the Ogryns, Ratlings and Squats. The founder and ruler of the Imperium is the god-like Emperor of Mankind, the most powerful Human psyker ever born. The Emperor founded the Imperium over 10,000 Terran years ago in the late 30th Millennium during the Unification Wars on Old Earth following the apocalyptic period in Human history known as the Age of Strife.


In [13]:
# Tokenize the text
encoded_summariser_prompt = tokenizer(summariser_prompt, return_tensors='pt').input_ids


In [14]:
# Generate summary with model 
config = GenerationConfig(
    do_sample = True,
    # Bigger value flattens the curve
    temperature = 1.0,
    #top_p = 0.8 # Top probabilities worth XX%
    top_k = 3 # Top X headlines
)

encoded_summary = summariser_model.generate(encoded_summariser_prompt, generation_config=config)


In [15]:
# Decode the summary
decoded_summary = tokenizer.decode(encoded_summary[0], skip_special_tokens=True)
print(decoded_summary)

The Imperium of Man, also known as the Imperium of Man, is a
